# Diluted Unitary Fitting with KL-Div

In [4]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

from quantum_tools import generate_unitary
from quantum_channel import channel_spectrum
from loss_functions import SpectrumDistance
from optimization import ModelQuantumMap, Logger
from kraus_channels import KrausMap, DilutedKrausMap
from utils import saver
from spectrum import mean_spacing
from utils import loader

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [5]:
def optimize_spectral_distance(spectrum_target = None, 
                              d = None, 
                              c_start = 0.5,
                              rank_start = 1,
                              rank_step = 1,
                              num_iter = 1000,
                              lr = 0.001,
                              verbose = True): 
    
    model_list = []
    distance_list = []
    counter = 0

    sigma = mean_spacing(spectrum_target)
    
    for rank in range(rank_start, d**2+rank_step, rank_step): 
        model = ModelQuantumMap(channel = DilutedKrausMap(U=generate_unitary(d), 
                                                          c=c_start, 
                                                          kraus_part=KrausMap(d, 
                                                                              rank=rank, 
                                                                              trainable=False),
                                                     ),
                            loss_function = SpectrumDistance(sigma=sigma, k = None, remove_shift=True),
                            optimizer = tf.optimizers.Adam(learning_rate=lr),
                            logger = Logger(loss_function = SpectrumDistance(sigma=sigma, k = None), 
                                            verbose=verbose,
                                            N=0),
                            )
        model.train(inputs = None,
                    targets = [spectrum_target],
                    num_iter = num_iter,
                    verbose = verbose,
                   )

        c_start = model.channel.c.numpy()
        model_list.append(model)
        distance_list.append(model.logger.loss_train_list[-1])
        
        idx = np.argmin(distance_list)
        if distance_list[-1] > distance_list[idx]:
            counter += 1
        else:
            counter = 0
            
        if counter > 3:
            print("Done!")
            break

    distance_best = distance_list[idx]
    model_best = model_list[idx]

    return distance_best, model_best

def fit_diluted_unitary(spectrum_target_list, 
                        d, 
                        rank_start, 
                        rank_step, 
                        num_iter = 1000,
                        lr = 0.001):
    
    distance_best_list = []
    model_best_list = []
    for spectrum_target in tqdm(spectrum_target_list):
        distance_best, model_best = optimize_spectral_distance(spectrum_target,
                                                              d,
                                                              rank_start,
                                                              rank_step,
                                                              num_iter = num_iter,
                                                              lr = lr)
        distance_best_list.append(distance_best)
        model_best_list.append(model_best)
    
    for model in model_best_list:
        model.optimizer = None
        
    return distance_best_list, model_best_list

## Four Qubits, Repetition

In [6]:
n = 4
d = 2**n

path1 = "../experiments/data/"
path2 = "data/"

[model1, model2, model3, model4, model5] = loader(path1 + "belem_4qubit_repeated.model")

spectrum1 = channel_spectrum(model1.channel, keep_unity=False)
spectrum2 = channel_spectrum(model2.channel, keep_unity=False) 
spectrum3 = channel_spectrum(model3.channel, keep_unity=False)
spectrum4 = channel_spectrum(model4.channel, keep_unity=False)
spectrum5 = channel_spectrum(model5.channel, keep_unity=False)

In [8]:
distance_best, model_best = optimize_spectral_distance(spectrum_target = spectrum1, 
                                                      d = d,
                                                      c_start = 0.5,
                                                      rank_start = 6, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=True)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_KL_4qubit_8layer.model")

  0%|          | 0/200 [00:00<?, ?it/s]

0.024331503714586023 None
-0.015304295830178186 None
-0.015305995886644483 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.017430603142703123 None
-0.017440709640181323 None
-0.017440709693987845 None


  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

-0.018989794221484314 None
-0.01901487947150083 None
-0.019014879857456785 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.020070412473068656 None
-0.02008508235167602 None


KeyboardInterrupt: 

In [ ]:
distance_best, model_best = optimize_spectral_distance(spectrum_target = spectrum2, 
                                                      d = d,
                                                      c_start = 0.5,
                                                      rank_start = 10, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_KL_4qubit_16layer.model")

  0%|          | 0/200 [00:00<?, ?it/s]

0.04533811068121134 None
0.045329996617866984 None
0.0453299964368201 None


  0%|          | 0/200 [00:00<?, ?it/s]

0.048500407705357323 None
-0.004775618224579954 None
-0.0050628142558500275 None


  0%|          | 0/200 [00:00<?, ?it/s]

...

-0.024810470474832036 None
-0.024823852118013703 None
-0.02482385386643443 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.02270932018871931 None
-0.023362337724488737 None
-0.023362361145985916 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.022285456585814212 None
-0.022615400341474218 None
-0.022615407816214343 None
Done!


In [ ]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_spectral_distance(spectrum_target = spectrum3, 
                                                      d = d, 
                                                      c_start = 0.18,
                                                      rank_start = 40, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=True)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_KL_4qubit_24layer.model")

  0%|          | 0/200 [00:00<?, ?it/s]

-0.009984272049604045 None
-0.015264161447782622 None
-0.015264171334487528 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.013456041423519222 None
-0.015784388754267388 None
-0.01578442147142492 None


  0%|          | 0/200 [00:00<?, ?it/s]

...

-0.024435826185256602 None
-0.024453200951644452 None
-0.024453204241121615 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.02265450407977239 None
-0.022681747009569493 None
-0.02268174712055275 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.021485065906558506 None
-0.02296371545447011 None
-0.02296383567084083 None
Done!


In [7]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_spectral_distance(spectrum_target = spectrum4, 
                                                      d = d,
                                                      c_start = 0.10, 
                                                      rank_start = 90, 
                                                      rank_step = 1,
                                                      num_iter = 300,
                                                      lr=0.01,
                                                      verbose=True)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_KL_4qubit_32layer.model")

  0%|          | 0/300 [00:00<?, ?it/s]

-0.038100598643609956 None
-0.03843560667600103 None
-0.038435607219506465 None
-0.03843560722015986 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.03869022950929066 None
-0.039377669512004845 None
-0.03937766951610727 None
-0.03937766951659376 None


  0%|          | 0/300 [00:00<?, ?it/s]

...

-0.04002664013726194 None
-0.04020657337276459 None
-0.04020657883922614 None
-0.04020657883923055 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.036898571298568425 None
-0.037566538728114454 None
-0.03756653889679909 None
-0.037566538896801255 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.03922052323234625 None
-0.03931511267746049 None
-0.03931511398207571 None
-0.03931511398227219 None
Done!


In [ ]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_spectral_distance(spectrum_target = spectrum5, 
                                                      d = d,
                                                      c_start = 0.10, 
                                                      rank_start = 100, 
                                                      rank_step = 1,
                                                      num_iter = 300,
                                                      lr=0.01,
                                                      verbose=True)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_KL_4qubit_40layer.model")

  0%|          | 0/300 [00:00<?, ?it/s]

-0.004737941247782984 None
-0.03787279979919901 None
-0.0378798164529017 None
-0.03787981649660596 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.03970895229089666 None
-0.039711740827375566 None
-0.0397117430677994 None
-0.039711743067856534 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.03541919551430274 None
-0.03545701244014264 None
-0.03545701322359257 None
-0.03545701322362371 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.03778877467933171 None
-0.03797028674791407 None
-0.037970298980582004 None
-0.03797029898058932 None
Done!
